<a href="https://colab.research.google.com/github/snsn3/Gen_AI_Crime_Prediction_Research_Article/blob/main/crime_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install required libraries: we will use GB from SK Learn
!pip install pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Load the training data
#this training set was prepared by randomly extracting a representative sample from the main file "Arrests and Strip Searches (RBDC-ARR-TBL-001), only for the year 2021"
train_df = pd.read_csv('training_set.csv')

# Replace missing categories with 'Unknown or Legacy' - some entries were missing at the time of dowloading the dataset
train_df['Occurrence_Category'] = train_df['Occurrence_Category'].fillna('Unknown or Legacy')

# Split the data into features (X) and labels (y)
X = train_df[['Occurrence_Category']]
y = train_df['Perceived_Race']

# One-hot encode the categorical features
ohe = OneHotEncoder(handle_unknown='ignore')
X_encoded = ohe.fit_transform(X)

# Encode the categorical labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Create a Gradient Boosting Classifier
gbc = GradientBoostingClassifier()

# Train the model on the training data
gbc.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = gbc.predict(X_val)

# Evaluate the model's performance
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation accuracy: {accuracy:.3f}')

# Load the data to predict
predict_df = pd.read_csv('empty_race_field_2021_ML.csv')

# Replace missing categories with 'Unknown or Legacy'
predict_df['Occurrence_Category'] = predict_df['Occurrence_Category'].fillna('Unknown or Legacy')

# Ensure that unknown categories are replaced with 'Unknown or Legacy'
known_categories = ohe.categories_[0]
predict_df['Occurrence_Category'] = predict_df['Occurrence_Category'].apply(
    lambda x: x if x in known_categories else 'Unknown or Legacy'
)

# One-hot encode the categorical features
predict_encoded = ohe.transform(predict_df[['Occurrence_Category']])

# Make predictions on the data
predictions = gbc.predict(predict_encoded)

# Decode the predictions to get the original labels (racial identities)
predicted_labels = le.inverse_transform(predictions)

# Fill the predicted labels into the existing empty column "Perceived_Race" - this was intentionally left blank in the file
#users may simply choose to create it from scract, if not needing to create and manually leave a blank column - users' choice
predict_df['Perceived_Race'] = predicted_labels

# Save the updated DataFrame back to the same CSV file
 #(the file we transformed after download from T. Police website)
predict_df.to_csv('ML_predicted_2021_test.csv', index=False)


Validation accuracy: 0.172


Visualizastion

In [ ]:
df=pd.read_csv ("ML_predicted_2021_test")
ml